In [1]:
import yaml
with open('prompts.yaml','r') as stream:
    prompt_templates=yaml.safe_load(stream)


In [2]:
# 使用rag
from langchain.vectorstores import FAISS
from tools.Retriever_tool import RetrieverTool
from langchain.embeddings import HuggingFaceEmbeddings
vectordb = FAISS.load_local(
    "./vectordb",
    embeddings=HuggingFaceEmbeddings(model_name="thenlper/gte-small"),
    allow_dangerous_deserialization=True,
)
retrievertool = RetrieverTool(vectordb=vectordb)

e:\Anaconda\envs\agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\aesc\AppData\Local\Temp\ipykernel_10036\2324574762.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="thenlper/gte-small"),


In [12]:
#知识检索 agent
from smolagents import CodeAgent,OpenAIServerModel,tool,HfApiModel,DuckDuckGoSearchTool
from tools.final_answer import FinalAnswerTool
from tools.visit_webpage import VisitWebpageTool
from tools.vision_comprehension import VisionComprehension
from tools.file_io import file_writer
model=OpenAIServerModel(model_id='qwen-max',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
qwen_agent=CodeAgent(model=model,
                     tools=[DuckDuckGoSearchTool(),file_writer(),retrievertool,VisitWebpageTool()],
                     additional_authorized_imports=['flask','os','matplotlib','pandas','numpy','seaborn','sklearn','torch','transformers','tensorflow','keras','cv2','PIL','matplotlib.pyplot','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL','matplotlib.pyplot as plt','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL'],
                     prompt_templates=prompt_templates,
                     name='qwen_agent',
                     description='information extraction agent,can do rag and web search')


In [13]:
#管理agent

from smolagents import CodeAgent,OpenAIServerModel,tool,HfApiModel,DuckDuckGoSearchTool
model=OpenAIServerModel(model_id='deepseek-v3',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", prtogether")
manage_agent=CodeAgent(model=model,
                       tools=[DuckDuckGoSearchTool(),file_writer(),FinalAnswerTool(),retrievertool],
                       additional_authorized_imports=['flask','os','matplotlib','pandas','numpy','seaborn','sklearn','torch','transformers','tensorflow','keras','cv2','PIL','matplotlib.pyplot','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL','matplotlib.pyplot as plt','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL'],
                       prompt_templates=prompt_templates,
                       max_steps=15,
                       planning_interval=5,
                       managed_agents=[qwen_agent]
                       )

In [14]:
manage_agent.visualize()

CodeAgent | deepseek-v3
├── ✅ Authorized imports: ['flask', 'os', 'matplotlib', 'pandas', 'numpy', 'seaborn', 'sklearn', 'torch', 
│   'transformers', 'tensorflow', 'keras', 'cv2', 'PIL', 'matplotlib.pyplot', 'matplotlib.pyplot as plt', 'pandas 
│   as pd', 'numpy as np', 'seaborn as sns', 'sklearn as sk', 'torch as torch', 'transformers as transformers', 
│   'tensorflow as tf', 'keras as keras', 'cv2 as cv2', 'PIL as PIL', 'matplotlib.pyplot as plt', 
│   'matplotlib.pyplot as plt', 'pandas as pd', 'numpy as np', 'seaborn as sns', 'sklearn as sk', 'torch as torch',
│   'transformers as transformers', 'tensorflow as tf', 'keras as keras', 'cv2 as cv2', 'PIL as PIL']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
│   │              │ your query (think a Google search) then       │ perform.                                     │
│   │              │ returns the top search results.               │                                              │
│   │ file_writer  │ you can use this tool to write to a simple    │ filename (`string`): the name of the file    │
│   │              │ file                                          │ content (`string`): the content to write to  │
│   │              │                                               │ the file                                     │
│   │              │                                               │ mode (`boolean`): the mode to write to the   │
│   │              │                                               │ file, if true, the file will be appended, if │
│   │              │                                               │ false, the file will be overwritten          │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   │ retriever    │ Using semantic similarity, retrieves some     │ query (`string`): The query to perform. This │
│   │              │ documents from the knowledge base that have   │ should be semantically close to your target  │
│   │              │ the closest embeddings to the input query.    │ documents. Use the affirmative form rather   │
│   │              │                                               │ than a question.                             │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── qwen_agent | CodeAgent | qwen-max
        ├── ✅ Authorized imports: ['flask', 'os', 'matplotlib', 'pandas', 'numpy', 'seaborn', 'sklearn', 'torch', 
        │   'transformers', 'tensorflow', 'keras', 'cv2', 'PIL', 'matplotlib.pyplot', 'matplotlib.pyplot as plt', 
        │   'pandas as pd', 'numpy as np', 'seaborn as sns', 'sklearn as sk', 'torch as torch', 'transformers as 
        │   transformers', 'tensorflow as tf', 'keras as keras', 'cv2 as cv2', 'PIL as PIL', 'matplotlib.pyplot as 
        │   plt', 'matplotlib.pyplot as plt', 'pandas as pd', 'numpy as np', 'seaborn as sns', 'sklearn as sk', 
        │   'torch as torch', 'transformers as transformers', 'tensorflow as tf', 'keras as keras', 'cv2 as cv2', 
        │   'PIL as PIL']
        ├── 📝 Description: information extraction agent,can do rag and web search
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                          

In [16]:
manage_agent.run(
    "这是一个数据分析任务,对于这个任务你需要分步来完成，所以对于管理agent首先你需要规划好整个流程，每一步需要怎么做，如有需要请利用检索agent检索信息。具体任务：'./data/anime-dataset-2023.csv' 这是一个与动漫相关的数据集，请自由发挥，进行数据挖掘，保存必要的图表与结论，最后帮我整理为一份数据分析报告,所有结果保存在'./results' 目录"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 这是一个数据分析任务,对于这个任务你需要分步来完成，所以对于管理agent首先你需要规划好整个流程，每一步需要怎么做  │
│ ，如有需要请利用检索agent检索信息。具体任务：'./data/anime-dataset-2023.csv'                                    │
│ 这是一个与动漫相关的数据集，请自由发挥，进行数据挖掘，保存必要的图表与结论，最后帮我整理为一份数据分析报告,所有 │
│ 结果保存在'./results' 目录                                                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-v3 ───────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. **Data Loading and Cleaning**  
   - Load the dataset `./data/anime-dataset-2023.csv` into a DataFrame.  
   - Inspect the dataset for missing values, duplicates, and inconsistencies.  
   - Clean the data by handling missing values, removing duplicates, and correcting inconsistencies.  
   - Standardize data formats (e.g., date formats, categorical encoding) as needed.  

2. **Exploratory Data Analysis (EDA) and Visualization**  
   - Perform univariate analysis to understand the distribution of individual variables (e.g., ratings, genres, 
episode counts).  
   - Conduct bivariate analysis to explore relationships between variables (e.g., correlation between ratings and 
popularity).  
   - Visualize key insights using appropriate charts (e.g., bar plots, scatter plots, heatmaps).  
   - Save all visualizations to the `./results` directory.  

3. **Statistical Analysis or Predictive Modeling**  
   - Perform statistical tests to identify significant relationships or trends (e.g., ANOVA, chi-square tests).  
   - If applicable, build predictive models (e.g., regression, classification) to forecast outcomes or classify 
data points.  
   - Evaluate model performance using appropriate metrics (e.g., RMSE, accuracy).  

4. **Interpretation of Results**  
   - Summarize key findings from the EDA and statistical analysis.  
   - Interpret the results in the context of the anime industry, providing actionable insights.  
   - Highlight limitations of the analysis and suggest areas for further investigation.  

5. **Report Compilation**  
   - Compile the analysis results, visualizations, and interpretations into a comprehensive report.  
   - Save the report to the `./results` directory in a readable format (e.g., PDF, Markdown).  

<end_plan
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import os                                                                                                        
                                                                                                                   
  # Load the dataset                                                                                               
  data = pd.read_csv('./data/anime-dataset-2023.csv')                                                              
                                                                                                                   
  # Inspect the dataset                                                                                            
  print("Dataset Info:")                                                                                           
  print(data.info())                                                                                               
  print("\nFirst 5 rows of the dataset:")                                                                          
  print(data.head())                                                                                               
  print("\nMissing values in each column:")                                                                        
  print(data.isnull().sum())                                                                                       
  print("\nDuplicate rows:")                                                                                       
  print(data.duplicated().sum())                                                                                   
                                                                                                                   
  # Handle missing values                                                                                          
  data.dropna(inplace=True)                                                                                        
                                                                                                                   
  # Remove duplicates                                                                                              
  data.drop_duplicates(inplace=True)                                                                               
                                                                                                                   
  # Save the cleaned dataset                                                                                       
  os.makedirs('./results', exist_ok=True)                                                                          
  data.to_csv('./results/cleaned_anime_dataset.csv', index=False)                                                  
                                                                                                                   
  print("\nData cleaning completed and saved to './results/cleaned_anime_dataset.csv'")                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non-null  object
 19  Popu

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 12 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(answer="The anime dataset analysis has been completed. The cleaned dataset, visualizations, and a   
  comprehensive report have been saved to the './results' directory. The report includes key insights,             
  statistical analysis, trend analysis, clustering results, and actionable recommendations.")                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The anime dataset analysis has been completed. The cleaned dataset, visualizations, and a 
comprehensive report have been saved to the './results' directory. The report includes key insights, statistical 
analysis, trend analysis, clustering results, and actionable recommendations.

[Step 12: Duration 30.44 seconds| Input tokens: 154,021 | Output tokens: 7,865]

"The anime dataset analysis has been completed. The cleaned dataset, visualizations, and a comprehensive report have been saved to the './results' directory. The report includes key insights, statistical analysis, trend analysis, clustering results, and actionable recommendations."

In [6]:
model=OpenAIServerModel(model_id='deepseek-v3',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
anaylse_agent=CodeAgent(
    model=model,
    tools=[
        RetrieverTool(vectordb=vectordb),
        VisitWebpageTool(),
        DuckDuckGoSearchTool(),
        VisionComprehension(),
        file_writer(),
        FinalAnswerTool(),
    ],
    max_steps=15,
)

In [7]:
anaylse_agent.run("请利用'./temp' 目录下面的文件（包括图片和txt），帮我写一份数据分析报告")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 请利用'./temp' 目录下面的文件（包括图片和txt），帮我写一份数据分析报告                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-v3 ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import os                                                                                                        
                                                                                                                   
  # 列出./temp目录下的所有文件                                                                                     
  files = os.listdir('./temp')                                                                                     
  print(files)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Warning to user: Code execution failed due to an unauthorized import - Consider passing said import under 
`additional_authorized_imports` when initializing your CodeAgent.

Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports
are: ['stat', 'unicodedata', 'statistics', 'time', 'collections', 'queue', 'itertools', 'math', 're', 'datetime', 
'random']

...

Execution logs:

Retrieved documents:
===== Document 0 =====
1.1.2 目标===== Document 1 =====
temp = []  
if count <=23:  
temp.append(i['od21'])     # 添加时间 
temp.append(i['od22'])     # 添加当前时刻温度  
temp.append(i['od24'])     # 添加当前时刻风力方向  
temp.append(i['od25'])     # 添加当前时刻风级 
temp.append(i['od26'])     # 添加当前时刻降水量  
temp.append(i['od27'])     # 添加当前时刻相对湿度===== Document 2 =====
如Python、Perl、R或Java）或UNIX文本处理工具（如sed或awk===== Document 3 =====
帮助用户更好地规划和选择旅行。首先，数据采集的目标是获取尽可能多的酒店信息。通过===== Document 4 =====
temp.append(i['od28'])     # 添加当前时刻控制质量  
#print(temp)  
final_day.append(temp)  
count = count +1
下面爬取7天的数据===== Document 5 =====
tem_low = inf[1].find('i').string   # 找到最低气温===== Document 6 =====
if inf[1].find('span') is None:   # 天气预报可能没有最高气温  
        tem_high = None  
    else:  
        tem_high = inf[1].find('span').string  # 找到最高气温  
    temp.append(tem_low[:-1])  
    if tem_high[-1] == '℃':  
     temp.append(tem_high[:-1])  
    else:  
     temp.append(tem_high)  
    wind = inf[2].find_all('span')  # 找到风向  
    for j in wind:

Out: None

[Step 1: Duration 6.70 seconds| Input tokens: 5,126 | Output tokens: 165]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 2: Duration 23.83 seconds| Input tokens: 7,814 | Output tokens: 253]

KeyboardInterrupt: 